# PyPetaKit5D Examples for Non-Light-Sheet Modalities

## Install PyPetaKit5D in your env

In [ ]:
# The installation may take a long time
# You can skip this step if you've already installed it in the env you are using for this notebook
%pip install --no-binary :all: --no-cache-dir PyPetaKit5D

## Download the 2P, Confocal, Phase, and Widefield Demo Datasets

In [ ]:
# You can set the folder where our demo dataset will be downloaded to
# By default it will be the Downloads folder in your home directory
# I.E. Setting petakit5d_demo_dataset_install_dir = "~/Downloads/" will install
# the dataset to ~/Downloads/PetaKit5D_2P_Confocal_Phase_Widefield_demo_datasets/
import os
import subprocess
import urllib.request

# Change this variable to your desired download location
petakit5d_demo_dataset_install_dir = "~/Downloads/"

petakit5d_demo_dataset_install_dir = os.path.expanduser(petakit5d_demo_dataset_install_dir)
petakit5d_demo_dataset_install_dir = os.path.join(petakit5d_demo_dataset_install_dir,'')
petakit5d_demo_dataset_url = "https://www.dropbox.com/scl/fi/sp1pz423jw6gpozab5yrf/PetaKit5D_2P_Confocal_Phase_Widefield_demo_datasets.tar?rlkey=i4wrwvphm9hz78z3kcqomqm1b&st=az3w6z0v&dl=1"
petakit5d_demo_dataset_dir_loc = os.path.join(petakit5d_demo_dataset_install_dir, "PetaKit5D_2P_Confocal_Phase_Widefield_demo_datasets/")
petakit5d_demo_dataset_tar_loc = os.path.join(petakit5d_demo_dataset_install_dir, "PetaKit5D_2P_Confocal_Phase_Widefield_demo_datasets.tar")
os.makedirs(os.path.dirname(petakit5d_demo_dataset_install_dir), exist_ok=True)
if not os.path.exists(petakit5d_demo_dataset_dir_loc):
    print("Downloading the demo dataset...")
    urllib.request.urlretrieve(petakit5d_demo_dataset_url, petakit5d_demo_dataset_tar_loc)
    print("Demo dataset downloaded successfully!")
    print("Untarring the demo dataset...")
    process = subprocess.Popen(
    f"tar -xf \"{petakit5d_demo_dataset_tar_loc}\" -C \"{petakit5d_demo_dataset_install_dir}\"", shell=True)
    process.wait()
    print("Demo dataset untarred successfully!")
    print("Removing the tar file...")
    os.remove(petakit5d_demo_dataset_tar_loc)
    print("Finished!")
    print(f"The demo dataset is located here: {petakit5d_demo_dataset_dir_loc}")
else:
    print(f"The demo dataset is already located here: {petakit5d_demo_dataset_dir_loc}")


## Generate an image list for Phase images from tile positions

In [ ]:
from PyPetaKit5D import XR_generate_image_list_wrapper

# We also include the image list csv file in
# {destPath}/PetaKit5D_2P_Confocal_Phase_Widefield_demo_datasets/Phase/ImageList_all_timepoints.csv

# result file:
# {destPath}/PetaKit5D_2P_Confocal_Phase_Widefield_demo_datasets/Phase/ImageList_from_tile_positions.csv

# Set your data path to our demo dataset
dataPath = '~/Downloads/PetaKit5D_2P_Confocal_Phase_Widefield_demo_datasets/'
currDataPath = [dataPath+'/Phase/']

# generation method
generationMethod = 'tile_position'

# channel pattern to include for image list generation
channelPatterns = ['Scan']

# tile patterns for t, c, x, y, z to determine the time point, channel and
# tile indices. It must be a unique pattern for them. Typically, it should
# be a number combined with some string with or without underscore/dash.
# The program will use this pattern to determine the tile information for
# all the tiles with the given patterns. 
# time or channel patterns can be left as emtpy string if there is only one
# time point or one channel. For channel, it can also include cam, like CamA_ch0
tilePatterns = ['0000t', 'ch0', '000x', '000y', '000z']

# xy pixel size
# the pixel size for the deskew/rotate data in the demo is isotropic at 0.108 um
xyPixelSize = 0.108

# objective scan: scanning in the DS space
objectiveScan = False

# inverted objective scan: scanning in the DSR space. 
IOScan = True

# overlap size between tiles: axis order: yxz in pixels, xyz in um
overlapSize = [100, 100, 1]

# overlap size unit: pixel or um
overlapSizeType = 'pixel'

XR_generate_image_list_wrapper(currDataPath, generationMethod, xyPixelSize=xyPixelSize,
    channelPatterns=channelPatterns, tilePatterns=tilePatterns, overlapSize=overlapSize,
    overlapSizeType=overlapSizeType, objectiveScan=objectiveScan, IOScan=IOScan);


## Run stitching for the Phase data

In [ ]:
from PyPetaKit5D import XR_matlab_stitching_wrapper

# The data is 2d and the coordinates are in the stage coordinates without any conversion
# so we use the 2D stitching scheme

# result folder:
# {destPath}/PetaKit5D_2P_Confocal_Phase_Widefield_demo_datasets/Phase/matlab_stitch_2d_phase/

# Set your data path to our demo dataset
dataPath = '~/Downloads/PetaKit5D_2P_Confocal_Phase_Widefield_demo_datasets/'
currDataPath = [dataPath+'/Phase/ff_corrected/']

# image list path: csv file
ImageListFullpath = [dataPath+'Phase/ImageList_all_timepoints.csv']

# stitch dir string, inside currDataPath
resultDirName = 'matlab_stitch_2d_phase'

# stitch in DS space
DS = False
# stitch in Deskew/rotated space, if both DS and DSR false, stitch in skewed space
DSR = False

# objective scan: scanning in the DS space
objectiveScan = False

# inverted objective scan: scanning in the DSR space. 
IOScan = True

# xy pixel size
# the pixel size for the deskew/rotate data in the demo is isotropic at 0.108 um
xyPixelSize = 0.108

# scan direction. 
reverse = True

# check the setting files to see if a tile is flipped (bidirectional scan)
parseSettingFile = False

# axis order
axisOrder = 'x,y,z'

# channels to stitch
channelPatterns = ['CamB_ch0']

# stitch blending method, 'none': no blending, 'feather': feather blending
blendMethod = 'feather'

# cross correlation registration, if false, directly stitch by the coordinates.
xcorrShift = True
xcorrMode = 'primary'

# max allowed shift (in pixel) in xy axes between neighboring tiles in xcorr registration. 
xyMaxOffset = 100

# max allowed shift in z (in pixel) axis between neighboring tiles in xcorr registration. 
zMaxOffset = 10

# downsampling factors for overlap regions to calculate xcorr in xcorr registration. 
# with larger downsamplinf factors, the faster of the xcorr computing, yet
# lower accuracy will be. 
xcorrDownSample = [1, 1, 1]

# if true, save result as 16bit, if false, save as single
save16bit = True

# chunk size in zarr
blockSize = [512, 512, 1]

# batch size for stitching, should be multipler for block size
batchSize = [2048, 2048, 1]

# tile offset: counts add to the image, used when the image background is
# 0, to differentiate between image background and empty space. 
tileOffset = 0

# erode the edge for given number of pixels
edgeArtifacts = 0

# here we use processed data that has already been flatfield correcetd, so we leave the processing function as empty.
processFunPath = ''

# if true, use slurm cluster for the computing; otherwise, use local machine
parseCluster = False

XR_matlab_stitching_wrapper(currDataPath, ImageListFullpath, DS=DS, DSR=DSR,
    processedDirStr='', reverse=reverse, objectiveScan=objectiveScan, IOScan=IOScan,
    axisOrder=axisOrder, xcorrShift=xcorrShift, xcorrMode=xcorrMode, xyMaxOffset=xyMaxOffset,
    zMaxOffset=zMaxOffset, xcorrDownSample=xcorrDownSample, channelPatterns=channelPatterns,
    resampleType='isotropic', xyPixelSize=xyPixelSize, blendMethod=blendMethod,
    blockSize=blockSize, batchSize=batchSize, resultDirName=resultDirName,
    parseSettingFile=parseSettingFile, processFunPath=processFunPath, tileOffset=tileOffset,
    edgeArtifacts=edgeArtifacts, save16bit=save16bit, parseCluster=parseCluster)

## Generate an image list for 2P images from tile positions

In [ ]:
from PyPetaKit5D import XR_generate_image_list_wrapper

# result file:
# {destPath}/PetaKit5D_2P_Confocal_Phase_Widefield_demo_datasets/2P/ImageList_from_tile_positions.csv

# Set your data path to our demo dataset
dataPath = '~/Downloads/PetaKit5D_2P_Confocal_Phase_Widefield_demo_datasets/'
currDataPath = [dataPath+'/2P/']

# generation method
generationMethod = 'tile_position'

# channels to include for image list generation
channelPatterns = ['Tile']

# tile patterns for t, c, x, y, z to determine the time point, channel and
# tile indices. It must be a unique pattern for them. Typically, it should
# be a number combined with some string with or without underscore/dash.
# The program will use this pattern to determine the tile information for
# all the tiles with the given patterns. 
# time or channel patterns can be left as emtpy string if there is only one
# time point or one channel. For channel, it can also include cam, like CamA_ch0
tilePatterns = ['0000t', 'ch0', '000x', '000y', '000z']

# xy pixel size
# the pixel size for the deskew/rotate data in the demo is isotropic at 0.108 um
xyPixelSize = 0.108

# scan step size
dz = 0.5

# objective scan: scanning in the DS space
objectiveScan = False

# inverted objective scan: scanning in the DSR space. 
IOScan = True

# overlap size between tiles: axis order: yxz in pixels, xyz in um
overlapSize = [10, 10, 5]

# overlap size unit: pixel or um
overlapSizeType = 'um'

XR_generate_image_list_wrapper(currDataPath, generationMethod, xyPixelSize=xyPixelSize,
    dz=dz, channelPatterns=channelPatterns, tilePatterns=tilePatterns, overlapSize=overlapSize,
    overlapSizeType=overlapSizeType, objectiveScan=objectiveScan, IOScan=IOScan)

## Run stitching for the 2P image

In [ ]:
from PyPetaKit5D import XR_matlab_stitching_wrapper
from datetime import datetime
import os

# result folder:
# {destPath}/PetaKit5D_2P_Confocal_Phase_Widefield_demo_datasets/2P/matlab_stitch/

# Set your data path to our demo dataset
dataPath = '~/Downloads/PetaKit5D_2P_Confocal_Phase_Widefield_demo_datasets/'
currDataPath = [dataPath+'/2P/']

ImageListFullpath = [currDataPath[0]+'/ImageList_from_tile_positions.csv']

# stitch dir string, inside dataPath
resultDirName = 'matlab_stitch'

# stitch in DS space
DS = False
# stitch in Deskew/rotated space, if both DS and DSR false, stitch in skewed space
DSR = False

# objective scan: scanning in the DS space
objectiveScan = False

# inverted objective scan: scanning in the DSR space. 
IOScan = True

# scan direction. 
reverse = True

# intensally change xy pixelsize from 0.108 to 0.15 to count for the large
# overlaps between tiles. 
xyPixelSize = 0.15

# scan step size
dz = 0.5

# axis order: the axis order for x and z are actually inversed for this dataset
axisOrder = '-x,y,-z'

# primary channel for cross registration
primaryCh = 'ch0'

# channels to stitch
channelPatterns = ['ch0']

# stitch blending method, 'none': no blending, 'feather': feather blending
blendMethod = 'feather'

# cross correlation registration, if false, directly stitch by the coordinates.
xcorrShift = True

# max allowed shift (in pixel) in xy axes between neighboring tiles in xcorr registration. 
xyMaxOffset = 200

# max allowed shift in z (in pixel) axis between neighboring tiles in xcorr registration. 
zMaxOffset = 10

# downsampling factors for overlap regions to calculate xcorr in xcorr registration. 
# with larger downsamplinf factors, the faster of the xcorr computing, yet
# lower accuracy will be. 
xcorrDownsample = [1, 1, 1]

# grid: only consider registration with direct neighbors
shiftMethod = 'grid'

# if true, save result as 16bit, if false, save as single
save16bit = False

# chunk size in zarr
blockSize = [256, 256, 256]

# batch size for stitching, should be multipler for block size
batchSize = [512, 512, 512]

# user defined processing function in tiff to zarr conversion, i.e., flat-field correction
# here is the example code to enable flat field correction
# define path to put the user defined function string (for the flat-field correction here).
tmpPath = currDataPath[0]+'/tmp/processFunc/'
tmpPath = os.path.expanduser(tmpPath)
os.makedirs(os.path.dirname(tmpPath), exist_ok=True)
# lower bound cap for flat-field
LowerLimit = 0.4
# if true, subtract background image from flat-field image first;
# otherwise, use flat-field image as it is.
FFBackground = False
# flat field image path
FFImagePaths = [currDataPath[0]+'/flatfield/flatfield.tif']
# background image path
backgroundPaths = [currDataPath[0]+'/flatfield/background.tif']
# offset to add after flat-field correction
tileOffset = 1
# erode given number of pixels in the edge
edgeArtifacts = 1

dt = datetime.now().strftime("%Y%m%d%H%M%S")
# number of channels
c = 1
# define user-defined function
usrFun = f"@(x)erodeVolumeBy2DProjection(processFFCorrectionFrame(x,'{FFImagePaths[c-1]}','{backgroundPaths[c-1]}',{tileOffset},{LowerLimit},{str(FFBackground).lower()}),{edgeArtifacts})"

# save user defined function to path
fn = f"{tmpPath}/processFunction_c1_{dt}.txt"
with open(fn, 'w') as file:
    file.write(usrFun)
processFunPath = [fn]

# use slurm cluster if true, otherwise use the local machine (master job)
parseCluster = False
# use master job for task computing or not. 
masterCompute = True
# configuration file for job submission
configFile = ''
# if true, use Matlab runtime (for the situation without matlab license)
mccMode = True

XR_matlab_stitching_wrapper(currDataPath, ImageListFullpath, DS=DS, DSR=DSR, processedDirStr='',
    reverse=reverse, objectiveScan=objectiveScan, IOScan=IOScan, axisOrder=axisOrder, 
    channelPatterns=channelPatterns, primaryCh=primaryCh, xcorrShift=xcorrShift, 
    xcorrDownsample=xcorrDownsample, shiftMethod=shiftMethod, xyMaxOffset=xyMaxOffset, 
    zMaxOffset=zMaxOffset, xyPixelSize=xyPixelSize, dz=dz, blendMethod=blendMethod, 
    resultDirName=resultDirName, processFunPath=processFunPath, save16bit=save16bit, 
    blockSize=blockSize, batchSize=batchSize, parseCluster=parseCluster, masterCompute=masterCompute, 
    configFile=configFile, mccMode=mccMode)

## Test the parameters for OMW backward projector for the Widefield image

In [ ]:
from PyPetaKit5D import XR_visualize_OTF_mask_segmentation

# Set your data path to our demo dataset
dataPath = '~/Downloads/PetaKit5D_2P_Confocal_Phase_Widefield_demo_datasets/'

psfFn = dataPath+'Widefield/WF_PSF.tif'
# OTF thresholding parameter
OTFCumThresh = 0.65
# true if the PSF is in skew space
skewed = False
# minimum internsity threshold for the OTF segmentation
minIntThrsh = 1e-3
XR_visualize_OTF_mask_segmentation(psfFn, OTFCumThresh, skewed, minIntThrsh=minIntThrsh)

## Run an OMW deconvolution for the Widefield image

In [ ]:
from PyPetaKit5D import XR_decon_data_wrapper

# Set your data path to our demo dataset
dataPath = '~/Downloads/PetaKit5D_2P_Confocal_Phase_Widefield_demo_datasets/'

# root path
rt = dataPath
# data path for data to be deconvolved, also support for multiple data folders
dataPaths = [rt+'/Widefield/']

# xy pixel size in um
xyPixelSize = 0.157
# z step size
dz = 0.1
# scan direction
reverse = True
# psf z step size (we assume xyPixelSize also apply to psf)
dzPSF = 0.1

# if true, check whether image is flipped in z using the setting files
parseSettingFile = False

# channel patterns for the channels, the channel patterns should map the
# order of PSF filenames.
channelPatterns = ['WF_raw']  

# psf path
psf_rt = rt            
psfFullpaths = [psf_rt+'/Widefield/WF_PSF.tif']           

# RL method
RLMethod = 'omw'
# wiener filter parameter
# alpha parameter should be adjusted based on SNR and data quality.
# typically 0.002 - 0.01 for SNR ~20; 0.02 - 0.1 or higher for SNR ~7
wienerAlpha = 0.02
# OTF thresholding parameter
OTFCumThresh = 0.65
# true if the PSF is in skew space
skewed = True
# hann window range applied to the distance transform, 0.0 means the center and 1.0 means border of OTF mask
hanWinBounds = [0.8, 1.2]
# deconvolution result path string (within dataPath)
resultDirName = 'matlab_decon_omw'

# background to subtract
background = 100
# number of iterations
deconIter = 20
# erode the edge after decon for number of pixels.
edgeErosion = 0
# save as 16bit; if false, save to single
save16bit = True
# use zarr file as input; if false, use tiff as input
zarrFile = False
# save output as zarr file; if false,s ave as tiff
saveZarr = False
# number of cpu cores
cpusPerTask = 24
# use cluster computing for different images
parseCluster = False
# set it to true for large files that cannot be fitted to RAM/GPU, it will
# split the data to chunks for deconvolution
largeFile = False
# use GPU for deconvolution
GPUJob = True
# if true, save intermediate results every 5 iterations.
debug = False
# config file for the master jobs that runs on CPU node
configFile = ''
# config file for the GPU job scheduling on GPU node
GPUConfigFile = ''
# if true, use Matlab runtime (for the situation without matlab license)
mccMode = True


# the results will be saved in matlab_decon under the dataPaths. 
# the next step is deskew/rotate (if in skewed space for x-stage scan) or 
# rotate (if objective scan) or other processings. 

# result folder:
# {destPath}/PetaKit5D_2P_Confocal_Phase_Widefield_demo_datasets/Widefield/matlab_decon_omw/

XR_decon_data_wrapper(dataPaths, resultDirName=resultDirName, xyPixelSize=xyPixelSize, dz=dz, reverse=reverse, 
    channelPatterns=channelPatterns, psfFullpaths=psfFullpaths, dzPSF=dzPSF, 
    parseSettingFile=parseSettingFile, RLMethod=RLMethod, wienerAlpha=wienerAlpha, 
    OTFCumThresh=OTFCumThresh, skewed=skewed, hanWinBounds=hanWinBounds, 
    background=background, deconIter=deconIter, edgeErosion=edgeErosion, 
    save16bit=save16bit, zarrFile=zarrFile, saveZarr=saveZarr, parseCluster=parseCluster, 
    largeFile=largeFile, GPUJob=GPUJob, debug=debug, cpusPerTask=cpusPerTask, 
    configFile=configFile, GPUConfigFile=GPUConfigFile, mccMode=mccMode)

## Test the parameters for OMW backward projector for the Confocal image

In [ ]:
from PyPetaKit5D import XR_visualize_OTF_mask_segmentation

# Set your data path to our demo dataset
dataPath = '~/Downloads/PetaKit5D_2P_Confocal_Phase_Widefield_demo_datasets/'
psfFn = dataPath+'/Confocal/Confocal_PSF.tif'
# OTF thresholding parameter
OTFCumThresh = 0.875
# true if the PSF is in skew space
skewed = False
# minimum internsity threshold for the OTF segmentation
minIntThrsh = 1e-3
XR_visualize_OTF_mask_segmentation(psfFn, OTFCumThresh, skewed, minIntThrsh=minIntThrsh);

## Run an OMW deconvolution for the Confocal image

In [ ]:
from PyPetaKit5D import XR_decon_data_wrapper

# Set your data path to our demo dataset
dataPath = '~/Downloads/PetaKit5D_2P_Confocal_Phase_Widefield_demo_datasets/'

# root path
rt = dataPath
# data path for data to be deconvolved, also support for multiple data folders
dataPaths = [rt+'/Confocal/']

# xy pixel size in um
xyPixelSize = 0.157
# z step size
dz = 0.1
# scan direction
reverse = True
# psf z step size (we assume xyPixelSize also apply to psf)
dzPSF = 0.1

# if true, check whether image is flipped in z using the setting files
parseSettingFile = False

# channel patterns for the channels, the channel patterns should map the
# order of PSF filenames.
channelPatterns = ['Confocal_raw']  

# psf path
psf_rt = rt            
psfFullpaths = [psf_rt+'/Confocal/Confocal_PSF.tif']            

# RL method
RLMethod = 'omw'
# wiener filter parameter
# alpha parameter should be adjusted based on SNR and data quality.
# typically 0.002 - 0.01 for SNR ~20; 0.02 - 0.1 or higher for SNR ~7
wienerAlpha = 0.004
# OTF thresholding parameter
OTFCumThresh = 0.875
# true if the PSF is in skew space
skewed = True
# deconvolution result path string (within dataPath)
resultDirName = 'matlab_decon_omw'

# background to subtract
background = 100
# number of iterations
deconIter = 3
# erode the edge after decon for number of pixels.
edgeErosion = 0
# save as 16bit; if false, save to single
save16bit = True
# use zarr file as input; if false, use tiff as input
zarrFile = False
# save output as zarr file; if false,s ave as tiff
saveZarr = False
# number of cpu cores
cpusPerTask = 24
# use cluster computing for different images
parseCluster = False
# set it to true for large files that cannot be fitted to RAM/GPU, it will
# split the data to chunks for deconvolution
largeFile = False
# use GPU for deconvolution
GPUJob = False
# if true, save intermediate results every 5 iterations.
debug = False
# config file for the master jobs that runs on CPU node
configFile = ''
# config file for the GPU job scheduling on GPU node
GPUConfigFile = ''
# if true, use Matlab runtime (for the situation without matlab license)
mccMode = False


# the results will be saved in matlab_decon under the dataPaths. 
# the next step is deskew/rotate (if in skewed space for x-stage scan) or 
# rotate (if objective scan) or other processings. 

# result folder:
# {destPath}/PetaKit5D_2P_Confocal_Phase_Widefield_demo_datasets/Confocal/matlab_decon_omw/

XR_decon_data_wrapper(dataPaths, resultDirName=resultDirName, xyPixelSize=xyPixelSize, dz=dz, reverse=reverse, 
    channelPatterns=channelPatterns, psfFullpaths=psfFullpaths, dzPSF=dzPSF, 
    parseSettingFile=parseSettingFile, RLMethod=RLMethod, wienerAlpha=wienerAlpha, 
    OTFCumThresh=OTFCumThresh, skewed=skewed, background=background,
    deconIter=deconIter, EdgeErosion=edgeErosion,  Save16bit=save16bit,
    zarrFile=zarrFile, saveZarr=saveZarr, parseCluster=parseCluster, 
    largeFile=largeFile, GPUJob=GPUJob, debug=debug, cpusPerTask=cpusPerTask, 
    ConfigFile=configFile, GPUConfigFile=GPUConfigFile, mccMode=mccMode)